In [1]:
"""Graphene lattice and LDOS demonstration script.

Usage example:
    python graphene_ldos_demo.py --nx 8 --ny 6 --orientation zigzag --outdir outputs/graphene
"""
from __future__ import annotations
import os


os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["VECLIB_MAXIMUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"] = "1"

import sys
from pathlib import Path
from collections import OrderedDict

import numpy as np
import matplotlib.pyplot as plt
import scipy.constants as spc
import scipy.sparse as sp
import matplotlib

matplotlib.use('Agg')
import matplotlib.pyplot as plt

_HERE = Path(__file__).resolve().parent if "__file__" in globals() else Path().resolve()
_SRC = _HERE.parent.parent
if str(_SRC) not in sys.path:
    sys.path.insert(0, str(_SRC))

from hamiltonian import Hamiltonian
from hamiltonian.geometry.graphene_generator import GrapheneGenerator
from negf.utils.block_partition import compute_optimal_block_sizes

In [10]:
ham = Hamiltonian(
    structure='graphene',
    nx=2,
    ny=2,
    graphene_orientation="armchair",
    a_cc=1.42,
    periodic_dirs='',
    graphene_passivate=False,
    passivate_x=False,
    comp_overlap=False,
).initialize()

In [11]:
h_matrix, hL0, hLC, hR0, hRC, h_periodic = ham.determine_leads()

In [12]:
print(h_matrix.shape)

(16, 16)


In [6]:
from negf.self_energy import surface_greens_function_nn
from negf.gf.recursive_greens_functions import _direct_inverse

In [ ]:
block_sizes = compute_optimal_block_sizes(ham.h_matrix, hLC)
energies = np.linspace(-5.0, 5.0, 400)
mu_left = 0.0
mu_right = 0.0

dos = []
for E in energies:
    Sigma_L = surface_greens_function_nn(E, hLC, hL0, hLC)[0]
    Sigma_R = surface_greens_function_nn(E,hRC, hR0, hRC)[-1]

    # Sigma_L =  hLC @ G00_L @ hLC.conj().T
    # Sigma_R = hRC @ G00_R @ hRC.conj().T

    G_R_diag, _, _, _, _ = _direct_inverse(E, h_matrix, Sigma_L, Sigma_R)

    dos.append(-np.imag(G_R_diag).sum() / np.pi)

plt.figure(figsize=(7, 5))
plt.plot(energies, dos)
plt.xlabel("Energy (eV)")
plt.ylabel("DOS (states/eV)")
plt.ylim((-.25, 5))
plt.title("DOS vs Energy")
plt.tight_layout()
plt.show()